# Курсовой проект для курса "Библиотеки Python для Data Science: Numpy, Matplotlib, Scikit-learn"

Материалы к проекту (файлы):
train.csv
test.csv

Задание:
Используя данные из train.csv, построить
модель для предсказания цен на недвижимость (квартиры).
С помощью полученной модели предсказать
цены для квартир из файла test.csv.

Целевая переменная:
Price

Метрика:
R2 - коэффициент детерминации (sklearn.metrics.r2_score)

Сдача проекта:
1. Прислать в раздел Задания Урока 10 ("Вебинар. Консультация по итоговому проекту")
ссылку на программу в github (программа должна содержаться в файле Jupyter Notebook 
с расширением ipynb). (Pull request не нужен, только ссылка ведущая на сам скрипт).
2. Приложить файл с названием по образцу SShirkin_predictions.csv
с предсказанными ценами для квартир из test.csv (файл должен содержать два поля: Id, Price).
В файле с предсказаниями должна быть 5001 строка (шапка + 5000 предсказаний).

Сроки и условия сдачи:
Дедлайн: сдать проект нужно в течение 72 часов после начала Урока 10 ("Вебинар. Консультация по итоговому проекту").
Для успешной сдачи должны быть все предсказания (для 5000 квартир) и R2 должен быть больше 0.6.
При сдаче до дедлайна результат проекта может попасть в топ лучших результатов.
Повторная сдача и проверка результатов возможны только при условии предыдущей неуспешной сдачи.
Успешный проект нельзя пересдать в целях повышения результата.
Проекты, сданные после дедлайна или сданные повторно, не попадают в топ лучших результатов, но можно узнать результат.
В качестве итогового результата берется первый успешный результат, последующие успешные результаты не учитываются.

Примечание:
Все файлы csv должны содержать названия полей (header - то есть "шапку"),
разделитель - запятая. В файлах не должны содержаться индексы из датафрейма.

Рекомендации для файла с кодом (ipynb):
1. Файл должен содержать заголовки и комментарии
2. Повторяющиеся операции лучше оформлять в виде функций
3. Не делать вывод большого количества строк таблиц (5-10 достаточно)
4. По возможности добавлять графики, описывающие данные (около 3-5)
5. Добавлять только лучшую модель, то есть не включать в код все варианты решения проекта
6. Скрипт проекта должен отрабатывать от начала и до конца (от загрузки данных до выгрузки предсказаний)
7. Весь проект должен быть в одном скрипте (файл ipynb).
8. При использовании статистик (среднее, медиана и т.д.) в качестве признаков,
лучше считать их на трейне, и потом на валидационных и тестовых данных не считать 
статистики заново, а брать их с трейна. Если хватает знаний, можно использовать кросс-валидацию,
но для сдачи этого проекта достаточно разбить данные из train.csv на train и valid.
9. Проект должен полностью отрабатывать за разумное время (не больше 10 минут),
поэтому в финальный вариант лучше не включать GridSearch с перебором 
большого количества сочетаний параметров.
10. Допускается применение библиотек Python и моделей машинного обучения,
которые были в курсе Python для Data Science. Градиентный бустинг изучается
в последующих курсах, поэтому в этом проекте его применять не следует.
Самая сложная из допустимых моделей - RandomForestRegressor из sklearn.

### Подключение библиотек и скриптов

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.metrics import r2_score as r2, accuracy_score
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
matplotlib.rcParams.update({'font.size': 14})

### 1. Чтение данных

In [4]:
def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):
    print("Train R2:\t" +str(round(r2(train_true_values, train_pred_values), 3)))
    print("Test R2:\t" +str(round(r2(test_true_values, test_pred_values), 3)))

    plt.figure(figsize=(18,10))
    
    plt.subplot(121)
    sns.scatterplot(x=train_pred_values, y=train_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Train sample prediction')
    
    plt.subplot(121)
    sns.scatterplot(x=test_pred_values, y=test_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Train sample prediction')

    plt.show()

In [5]:
def show_dst(train_df, column):
    plt.figure(figsize = (16, 8))
    
    train_df[column].hist(bins=18)
    plt.ylabel('Count')
    plt.xlabel(column)
    
    plt.title('Target distribution')
    plt.show()
    return None

### Пути к диреториям и файлам

In [6]:
TRAIN_DATASET_PATH = 'input.train.csv'
TEST_DATASET_PATH = 'input.test.csv'

In [7]:
sns.jointplot

<function seaborn.axisgrid.jointplot(*, x=None, y=None, data=None, kind='scatter', color=None, height=6, ratio=5, space=0.2, dropna=False, xlim=None, ylim=None, marginal_ticks=False, joint_kws=None, marginal_kws=None, hue=None, palette=None, hue_order=None, hue_norm=None, **kwargs)>

### Загрузка данных

**Описание датасета**

* **Id** - идентификационный номер квартиры
* **DistrictId** - идентификационный номер района
* **Rooms** - количество комнат
* **Square** - площадь
* **LifeSquare** - жилая площадь
* **KitchenSquare** - площадь кухни
* **Floor** - этаж
* **HouseFloor** - количество этажей в доме
* **HouseYear** - год постройки дома
* **Ecology_1, Ecology_2, Ecology_3** - экологические показатели местности
* **Social_1, Social_2, Social_3** - социальные показатели местности
* **Healthcare_1, Helthcare_2** - показатели местности, связанные с охраной здоровья
* **Shops_1, Shops_2** - показатели, связанные с наличием магазинов, торговых центров
* **Price** - цена квартиры

In [8]:
train_df = pd.read_csv(TRAIN_DATASET_PATH)
print(train_df.shape)
train_df.head()

(10000, 20)


,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


In [9]:
train_df.isna().sum

<bound method DataFrame.sum of          Id  DistrictId  Rooms  Square  LifeSquare  KitchenSquare  Floor  \
0     False       False  False   False       False          False  False   
1     False       False  False   False       False          False  False   
2     False       False  False   False       False          False  False   
3     False       False  False   False       False          False  False   
4     False       False  False   False       False          False  False   
...     ...         ...    ...     ...         ...            ...    ...   
9995  False       False  False   False       False          False  False   
9996  False       False  False   False       False          False  False   
9997  False       False  False   False        True          False  False   
9998  False       False  False   False       False          False  False   
9999  False       False  False   False       False          False  False   

      HouseFloor  HouseYear  Ecology_1  Ecology_2  Ecolo

In [10]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             10000 non-null  int64  
 1   DistrictId     10000 non-null  int64  
 2   Rooms          10000 non-null  float64
 3   Square         10000 non-null  float64
 4   LifeSquare     7887 non-null   float64
 5   KitchenSquare  10000 non-null  float64
 6   Floor          10000 non-null  int64  
 7   HouseFloor     10000 non-null  float64
 8   HouseYear      10000 non-null  int64  
 9   Ecology_1      10000 non-null  float64
 10  Ecology_2      10000 non-null  object 
 11  Ecology_3      10000 non-null  object 
 12  Social_1       10000 non-null  int64  
 13  Social_2       10000 non-null  int64  
 14  Social_3       10000 non-null  int64  
 15  Healthcare_1   5202 non-null   float64
 16  Helthcare_2    10000 non-null  int64  
 17  Shops_1        10000 non-null  int64  
 18  Shops_2

### Выбрасываем признаки

In [11]:
train_df.replace({'Ecology_2' : {'A':0, 'B':1}, 'Ecology_3' : {'A':0, 'B':1}, 'Shops_2' : {'A':0, 'B':1}}, inplace = True)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(train_df.drop(['LifeSquare', 'Healthcare_1', 'Price'], axis = 'columns'), 
                                           train_df['Price'], test_size = 0.2, random_state=22)

In [13]:
model_RFR = RandomForestRegressor(max_depth=15, n_estimators=200, random_state = 55)
model_RFR.fit(X_train, y_train)
first_predict = model_RFR.predict(X_test)
r2(y_test, first_predict)

0.7114703008091737

In [14]:
model_RFR.fit(train_df.drop(['LifeSquare', 'Healthcare_1', 'Price'], axis = 'columns'), train_df['Price'])

RandomForestRegressor(max_depth=15, n_estimators=200, random_state=55)

In [15]:
test_df = pd.read_csv(TEST_DATASET_PATH)

In [16]:
# Заменяем объекты в тесте аналогично трейну
test_df.replace({'Ecology_2' : {'A':0, 'B':1}, 'Ecology_3' : {'A':0, 'B':1}, 'Shops_2' : {'A':0, 'B':1}}, inplace = True)

In [17]:
test_df.isna().sum

<bound method DataFrame.sum of          Id  DistrictId  Rooms  Square  LifeSquare  KitchenSquare  Floor  \
0     False       False  False   False       False          False  False   
1     False       False  False   False        True          False  False   
2     False       False  False   False       False          False  False   
3     False       False  False   False       False          False  False   
4     False       False  False   False       False          False  False   
...     ...         ...    ...     ...         ...            ...    ...   
4995  False       False  False   False       False          False  False   
4996  False       False  False   False       False          False  False   
4997  False       False  False   False       False          False  False   
4998  False       False  False   False        True          False  False   
4999  False       False  False   False        True          False  False   

      HouseFloor  HouseYear  Ecology_1  Ecology_2  Ecolo

In [18]:
first_predict = model_RFR.predict(test_df.drop(['LifeSquare', 'Healthcare_1'], axis = 'columns'))

In [19]:
test_df['Price'] = first_predict

In [20]:
test_df[['Id', 'Price']].to_csv('Prediction3.csv', index = False)